<a href="https://colab.research.google.com/github/FSenciani/Dados-Python---Pandas-Matplotlib-Numpy./blob/main/MATERIAL_TESTE_DADOS_ENEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pandas matplotlib numpy seaborn

In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt

# Load Data


In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv('dados_enem_2021_BA.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dados_enem_2021_BA.csv'

In [ ]:
df.shape

No Pandas podemos carregar dados de diferentes formatos. Os principais são :

CSV
Excel
SQL
Usando PySpark um formato muito utilizado é o parquet. Veremos este ponto mais adiante.

De igual modo podemos escrever os arquivos em diferentes formatos (output).

Na prática o formato de dados que iremos trabalhar vai depender muito da origem dos dados, da estrutura da empresa que estaremos trabalhando, etc.


# Microdados ENEM

Grupos de variáveis

Dados do participante
Dados da escola
Dados do local de aplicação da prova
Dados da prova objetiva
Dados da redação
Dados do questionário socioeconômico

Desafio :Elaborar um gráfico de pareto com os estados e as maiores notas.

# Inspeção de Dados


Hipótese levantadas

Padrões espaciais afetam o desempenho escolar?
Características municipais afetam o desempenho escolar?
Renda familiar pode afetar a nota dos alunos?
Há diferença de desempenho entre escolar públicas e privadas?
Qual matéria tem maior nota naquela região?

In [ ]:
# leitura das 5 primeiras linhas
df.head()

In [ ]:
df.head(n = 10)

In [ ]:
# leitura das 5 últimas linhas
df.tail()

In [ ]:
df.tail(n = 10)

Nota: os comandos df.head() e df.tail() são úteis para inspecionar e compreender o dataset.

In [ ]:
# quantidade de linhas x quantidade de colunas
# o resultado é uma tupla - podemos verificar com a função type()
df.shape

In [ ]:
# Sumário do Dataset
# número de valores não nulos, dtype de cada coluna, memory usage
df.info()

In [ ]:
# verificar missing (dados ausentes)
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending = False)

In [ ]:
# nome das colunas
df.columns

In [ ]:
# Sumário estatístico dos dados
# 25%: Primeiro Quartil
# 50%: Segundo Quartil (Mediana)
# 75%: Terceiro Quartil
df.describe().transpose()

In [ ]:
df.index

In [ ]:
# Quantidade de valores únicos por coluna
df.nunique().sort_values(ascending = True)

In [ ]:
type(df)

In [ ]:
# seleciona uma coluna
df.TP_ESCOLA

In [ ]:
# distribuição de frequência
df.TP_ESCOLA.value_counts()

In [ ]:
# seleciona uma coluna
df['TP_ESCOLA']

In [ ]:
# seleciona uma coluna
# .loc[] : seleciona colunas pelo label
# sintaxe: df.loc[linhas:colunas]
df.loc[:, 'TP_SEXO']

In [ ]:
# seleciona múltiplas
df[['TP_SEXO', 'TP_ESCOLA']]

In [ ]:
# seleciona múltiplas colunas com .loc
df.loc[:, ['TP_SEXO', 'TP_ESCOLA']]

In [ ]:
# .iloc[] : seleciona colunas pela posição
# sintaxe: df.iloc[linhas:colunas]
df.iloc[:, 0]

In [ ]:
# exemplo de seleção com .iloc
df.iloc[0:5, 0]

Com a função .select_dtypes() podemos selecionar colunas de acordo com seu dtype (int, float, bool, etc.). Em várias situações isso pode ser útil, dentre outras coisas para aplicarmos rotinas específicas de acordo com o dtype ou ainda separar as colunas em qualitativas e quantitivas.

In [ ]:
# seleciona colunas numéricas
df.select_dtypes(include = [int, float])

In [ ]:
# outra forma de fazer:
df.select_dtypes(exclude = object)

In [ ]:
# seleciona colunas str
# df.select_dtypes(exclude = [int, float])
df.select_dtypes(include = object)

In [ ]:
cat_cols = df.select_dtypes(include = object).columns.tolist()
num_cols = df.select_dtypes(include = [int, float]).columns.tolist()

# Filtro

In [ ]:
# alunos de colégio público
df.query('TP_ESCOLA == 2').NU_NOTA_MT.mean()

In [ ]:
# alunos de colégio privado
df.query('TP_ESCOLA == 3').NU_NOTA_MT.mean()

In [ ]:
# alunos de colégio privado
df.query('TP_ESCOLA == 3').NU_NOTA_MT.agg([np.mean, np.median])

In [ ]:
# alunos de colégio pública
df.query('TP_ESCOLA == 2').NU_NOTA_MT.agg([np.mean, np.median])

In [ ]:
# 1 condição
df.query('TP_SEXO == "M"').sample(5)

In [ ]:
# validando a nossa consulta
# .unique(): retorna os valores únicos de uma determinada coluna
df.query('TP_SEXO == "M"')['TP_SEXO'].unique()

Agora vamos retornar um subset com os alunos do sexo Masculino que sejam treineiros. Neste caso usaremos o operador & (AND).

In [ ]:
df.query('(TP_SEXO == "M") & (IN_TREINEIRO == 1)')

In [ ]:
df.query("(TP_SEXO == 'M') & (IN_TREINEIRO == 1)").sample(5)

Se o objetivo fosse retornar os alunos do sexo masculino ou que foram treineiros, poderíamos usar o operador | (OR).

In [ ]:
df.query("(TP_SEXO == 'M') | (IN_TREINEIRO == 1)").sample(5)

Outra sintaxe:

In [ ]:
df.head()

In [ ]:
# cria uma máscara booleana
mask = df.TP_SEXO == 'M'
mask

In [ ]:
df[mask]

In [ ]:
df[df.TP_SEXO == "M"].sample(5)

In [ ]:
df.loc[df.TP_SEXO == "M"].sample(5)

In [ ]:
df[(df.TP_SEXO == "M") & (df.IN_TREINEIRO == 1)].sample(5)

In [ ]:
df.loc[(df.TP_SEXO == "M") & (df.IN_TREINEIRO == 1)].sample(5)

Uma forma alternativa:

In [ ]:
condition_1 = df.TP_SEXO == "M"
condition_2 = df.IN_TREINEIRO == 1
df.loc[(condition_1) & (condition_2)].sample(5)

Negação

In [ ]:
df[df.NO_MUNICIPIO_PROVA.isin(['Itabuna', 'Salvador'])].NO_MUNICIPIO_PROVA.unique()

Podemos usar o perador ~ para negar uma determinada condição:

In [ ]:
df[~(df.NO_MUNICIPIO_PROVA.isin(['Salvador']))]

# Visualização de Dados

In [ ]:
provas = df.columns[(df.columns.str.contains('NOTA')) & (~df.columns.str.contains('COMP'))].tolist()
idCandidato = ['NU_INSCRICAO']

Utilizando uma sintaxe simples da biblioteca Pandas conseguimos gerar análises gráficas rapidamente. Podemos fazer os seguintes tipos de gráficos:

* Barras
* Pizza
* Linha
* Histograma
* Boxplot
* Densidade
* Área
* Dispersão


Para obter mais detalhes acesse a documentação [aqui](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html).

In [ ]:
df.TP_ESCOLA.value_counts().plot(kind = 'bar')
plt.title('Tipo escola dos participantes no ENEM 2021')

In [ ]:
df.TP_SEXO.value_counts().plot(kind = 'bar')

Melhorando um pouco o plot:

In [ ]:
# gráfico de barras
df.TP_SEXO.value_counts().plot(kind = 'bar', color = 'black')
# título
plt.title('Proporção de gêneros no ENEM 2021')
# label do eixo x
plt.xlabel('Sexo')
# label do eixo y
plt.ylabel('Quantidade')

plt.show()
#plt.xlabel('Sexo');plt.ylabel('Quantidade');plt.show()

In [ ]:
#df.NO_MUNICIPIO_PROVA\
#.value_counts()\
#.reset_index()

In [ ]:
(
    df.NO_MUNICIPIO_PROVA
    .value_counts()
    .reset_index()
    .rename(columns = {'index' : 'Município', 'NO_MUNICIPIO_PROVA': 'Quantidade'})
    .nlargest(n = 15, columns = 'Quantidade')
    .set_index('Município')
    .plot(
        kind = 'barh',
        color = 'black',
        legend = False,
        xlabel = 'Quantidade',
        title = 'Municípios baianos com maior quantidade de inscritos no ENEM 2021'
    )
);

# Gráfico de pizza

* Proporção entre os gêneros

In [ ]:
df.TP_SEXO.value_counts().plot(kind = 'pie');

* Tipo escola

In [ ]:
(
    df.TP_ESCOLA.map({1:'Não respondeu', 2:'Pública', 3:'Privada'})
    .value_counts()
    .plot(kind = 'pie',
          legend = False,
          autopct = '%.2f'
         )
);

Gráfico de dispersão das notas de Matemática e Ciências da Natureza:

In [ ]:
df.plot(kind = 'scatter', x = 'NU_NOTA_MT', y = 'NU_NOTA_CN')

Histograma

In [ ]:
df.NU_NOTA_MT.plot(kind = 'hist')

Boxplot

In [ ]:
df.NU_NOTA_MT.plot(kind = 'box')

In [ ]:
df[provas].plot(kind = 'box')

Kde (densidade)

In [ ]:
df.NU_NOTA_MT.plot(kind = 'kde')

Gráfico de linhas

In [ ]:
(
    df
    .groupby(by = ['NO_MUNICIPIO_PROVA'])
    .agg({'NU_NOTA_MT': np.mean})
    .reset_index()
    .rename(columns = {'NU_NOTA_MT' : 'Nota', 'NO_MUNICIPIO_PROVA': 'Município'})
    .nlargest(n = 10, columns = 'Nota')
    .plot(
        x       = 'Município',
        y       = 'Nota',
        figsize = (15, 10),
        color   = 'k',
        ylabel  = 'Média em Matemática',
        title   = 'Nota média em Matemática por municípios',
        #ylim    = [520, 545]

    )

);

# Ordenação

Em alguns casos precisamos ordenar os dados por alguma coluna ou conjunto de colunas. No Pandas podemos utilizar a função .sort_values().

In [ ]:
provas = df.columns[(df.columns.str.contains('NOTA')) & (~df.columns.str.contains('COMP'))].tolist()
idCandidato = ['NU_INSCRICAO']

In [ ]:
df_subset = df[idCandidato+provas].sample(n = 10, random_state = 9999)

In [ ]:
# ordena o dataset em ordem decrescente
df_subset.sort_values(by = ['NU_NOTA_MT'], ascending = False)

In [ ]:
# ordena o dataset em ordem crescente
df_subset.sort_values(by = ['NU_NOTA_MT'], ascending = True)

Podemos definir onde queremos inserir os valores ausentes com na_position.

In [ ]:
df_subset.sort_values(by = ['NU_NOTA_MT'], na_position = 'first', ascending = False)

In [ ]:
df_subset.sort_values(by = ['NU_NOTA_MT'], na_position = 'last', ascending = False)

Podemos efetuar a ordenação por mais de uma coluna:

In [ ]:
df_subset.sort_values(by = ['NU_NOTA_MT', 'NU_NOTA_CN'], ascending = [False, False])

Podemos ainda aplicar a função .sort_values() para uma Series, neste caso não precisamos especificar o argumento by:

In [ ]:
type(df.NU_NOTA_MT)

In [ ]:
df_subset.NU_NOTA_MT.sort_values()

In [ ]:
df_subset.NU_NOTA_MT.sort_values(ascending = True, na_position = 'first')

Em alguns casos estamos interessados em criar um ranking. Vamos filtra a base para os candidatos que fizeram a prova em Salvador e posteriormente criar um top 10 na prova de Matemática.

In [ ]:
df_subset = df.query('NO_MUNICIPIO_PROVA == "Salvador"')[idCandidato+provas]

In [ ]:
df_subset

In [ ]:
df_subset.sort_values(by = ['NU_NOTA_MT'], ascending = False).iloc[:11, :]

Outra forma de fazer isso seria utilizando a função nlargest (para análise contrária podemos usar a função nsmallest):

In [ ]:
df_subset.nlargest(n = 10, columns = 'NU_NOTA_MT')

Podemos ainda analisar por mais de 1 coluna:

In [ ]:
df_subset.sort_values(by = ['NU_NOTA_MT', 'NU_NOTA_CN'], ascending = False).iloc[:10, :]

In [ ]:
# caso houvesse duplicidade poderíamos passar o argumento keep
# keep = first : take the first occurrence.
# keep = last : take the last occurrence.
# keep = all : do not drop any duplicates, even it means selecting more than n items.
df_subset.nlargest(n = 10, columns = ['NU_NOTA_MT', 'NU_NOTA_CN'], keep = 'first')

In [ ]:
df_subset.nsmallest(n = 10, columns = ['NU_NOTA_MT', 'NU_NOTA_CN'], keep = 'first')

Índice



A indexação do Pandas conta as linhas e colunas a partir do zero, mas podemos alterar o índice default do DataFrame e ainda ordenar um DataFrame de acordo com seu índice. Em séries temporais alterar o índice default pode facilitar o trabalho de filtro de datas.

* set_index()

In [ ]:
df_subset.set_index('NU_INSCRICAO', inplace = True)

In [ ]:
df_subset.head()

* sort_index()

In [ ]:
# ordena o dataset de acordo com o índice
df_subset.sort_index().head()

Podemos escolhar mais de uma coluna como índice do DataFrame, com efeito criamos um MultiIndex.

In [ ]:
df_test = df.copy().sample(10)
df_test = df_test.set_index(['NO_MUNICIPIO_PROVA', 'TP_SEXO'])

In [ ]:
df_test

In [ ]:
type(df_test.index)

* reset_index()

Para retornar ao dataset original:

In [ ]:
df_subset = df_subset.reset_index(drop = False)

In [ ]:
df_subset.head()

# Agregação/Sumarização

Nesta seção vamos aprender a aplicar medidas para sumarizar dados, tais como soma, contagem, média, mediana, etc.

Em geral para aplicar uma função de agregação em uma coluna a sintaxe será :

df.column.agg_function()

Função|Descrição
------|----------
df.sum()|Soma
df.count()|Contagem
df.abs()|Valor absoluto
df.cumsum()|Soma cumulativa
df.cumprod()|Produto cumulativo
df.min()|Mínimo
df.max()|Máximo
df.mean()|Média
df.median()|Mediana
df.mode()|Moda
df.var()|Variância
df.std()|Desvio Padrão
df.mad()|Desvio Médio Absoluto
df.skew()|Medida de Assimetria da Distribuição
df.kurtosis()|Curtose
df.corr()|Correlação
df.cov()|Covariância
df.quantile()|Quantil

Agora vejamos alguns exemplos:

In [ ]:
# média das provas
df[provas].mean()

In [ ]:
# moda da variável sexo
df['TP_SEXO'].mode()

A função .agg() permite aplicar mais de uma função de agregação ou operações ao longo dos eixos. Aggregate using one or more operations over the specified axis. Neste caso podemos inserir funções de agregação do NumPy.

In [ ]:
# valor máximo das provas
# df[provas].agg(max)
df[provas].agg(np.max)

Para aplicar mais de uma função de agregação podemos passar uma lista contendo todas as agregações desejadas:

In [ ]:
df[provas].agg([np.min, np.max, np.mean])

Podemos mudar o eixo de aplicação das funções de agregaçaõ. Isto é, podemos aplicar uma soma nas linhas ou nas colunas, por exemplo.

In [ ]:
# média das colunas
df[provas].mean(axis = 0)

In [ ]:
# média das linhas
df[provas].mean(axis = 1)

Observe que no primeiro caso nosso objetivo foi analisar a média geral dos alunos da amostra. No segundo caso, fizemos o cálculo da média de cada aluno nas provas.

Em alguns casos estamos interessados em obter o valor máximo, mas também obter a linha deste registro. Idem para o mínimo. Neste caso, podemos usar as funções idxmin() e idxmax(). Para séries temporais tal análise é importante, pois conseguimos obter, por exemplo, a data de ocorrência do valor máximo ou mínimo.

In [ ]:
# valor máximo, índice de ocorrência do valor máximo
df.NU_NOTA_MT.max(), df.NU_NOTA_MT.idxmax()

In [ ]:
# verifica o resultado anterior
df.iloc[df.NU_NOTA_MT.idxmax(), ]

# Transformação de dados

Nesta seção vamos aprender algumas técnicas de transformação dos dados.

In [ ]:
df_ssa = df[df.NO_MUNICIPIO_PROVA == 'Salvador'][idCandidato+provas].sample(10, random_state = 10)

Vamos criar duas novas colunas:

Média das provas Objetivas
Média Global
Vejamos algumas formas distintas de alcançar este objetivo.

In [ ]:
provas_objetivas = provas.copy()
provas_objetivas.remove('NU_NOTA_REDACAO')

Uma forma de fazer isso seria:

In [ ]:
df_ssa['NU_MEDIA_OBJ_1'] = (df_ssa['NU_NOTA_CN']+df_ssa['NU_NOTA_CH']+df_ssa['NU_NOTA_LC']+df_ssa['NU_NOTA_MT'])/4

In [ ]:
df_ssa['NU_MEDIA_OBJ_1']

Usando função de agregação:

In [ ]:
df_ssa[provas_objetivas].mean(axis = 1)

In [ ]:
df_ssa[provas_objetivas].agg(func = np.mean, axis = 1)

Uma pequena diferença se usarmos uma lista dentro do argumento func:

In [ ]:
df_ssa[provas_objetivas].agg(func = [np.mean], axis = 1)

Outra forma de fazermos isso seria utilizando a função pd.eval() ou ainda a função assign:

In [ ]:
df_ssa.eval('(NU_NOTA_CN+NU_NOTA_CH+NU_NOTA_LC+NU_NOTA_MT) / 4')

In [ ]:
df_ssa.assign(media_obj = (df_ssa.NU_NOTA_MT+df_ssa.NU_NOTA_CN+df_ssa.NU_NOTA_LC+df_ssa.NU_NOTA_CH) / 4)

Em algumas análises podemos tomar o log de uma determinada variável, veremos mais sobre isso na aula de Estatística Básica.

In [ ]:
np.log(df_ssa.NU_NOTA_MT)

In [ ]:
df_ssa = df_ssa.assign(
    log_nota_mt = lambda col: np.log(col.NU_NOTA_MT) ,
    log_nota_cn = lambda col: np.log(col.NU_NOTA_CN) ,
    log_nota_ch = lambda col: np.log(col.NU_NOTA_CH) ,
    log_nota_lc = lambda col: np.log(col.NU_NOTA_LC)
             )

Podemos usar ainda a função .apply():

In [ ]:
df_ssa[provas_objetivas].apply(lambda col: np.log(col))

Aplicando funções

map()

percorre item por item e aplica uma função desejada.
Neste exemplo, vamos percorrer cada valor de renda e aplicar a função np.log().

In [ ]:
df.query('NU_NOTA_MT != 0').NU_NOTA_MT.map(np.log)

Podemos usar a funçaõ map() para transformar variáveis. Na coluna TP_COR_RACA temos informações de cor/raça do candidato.

In [ ]:
# 0 : Não declarado
# 1 : Branca
# 2 : Preta
# 3 : Parda
# 4 : Amarela
# 5 : Indígena

In [ ]:
np.sort(df.TP_COR_RACA.unique())

In [ ]:
mapear_valores_cor_raca = {
    0 : 'Não declarado',
    1 : 'Branca',
    2 : 'Preta',
    3 : 'Parda',
    4 : 'Amarela',
    5 : 'Indígena'
}

In [ ]:
df.TP_COR_RACA.map(mapear_valores_cor_raca)

Podemos usar também a função replace:

In [ ]:
df.TP_SEXO.replace({'M': 'Masculino', 'F' : 'Feminino'})

apply()

podemos aplicar uma função em todas as linhas ou em todas as colunas (ou seja, aplica uma função em um determinado eixo).

In [ ]:
def resultado_candidato(nota):
    if nota < 600 :
        return 'Reprovado'
    else:
        return 'Aprovado'

In [ ]:
df[df.NU_NOTA_MT.notna()].NU_NOTA_MT.apply(resultado_candidato)

Podemos usar apply() para transformar uma coluna:

In [ ]:
def mapear_tipo_escola(tipo_escola):
    if tipo_escola == 1:
        return 'Não respondeu'
    elif tipo_escola == 2:
        return 'Pública'
    else:
        return 'Privada'

In [ ]:
df.TP_ESCOLA.apply(mapear_tipo_escola)

In [ ]:
df.TP_SEXO.apply(lambda col: 'Masculino' if col == 'M' else 'Feminino')

# Agrupamento

Anteriormente aprendemos a aplicar funções de agregação de forma genérica para uma ou mais variáveis. Algo muito como quando estamos analisando dados é obter tais medidas agrupadas. Ou seja, qual a média da prova de matemática por gênero?

<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png" height=400 width=400>

O flow é o seguinte:

split: os dados são separados de acordo com seus grupos (A, B, C).
apply: após o split aplica-se a função de agregação desejada, neste caso a função sum().
combine: o resultado final é combinado.
Então note que, após separar os grupos e aplicar a soma para cada grupo, obtemos um dataset final com o resultado desta operação (split-apply-combine).

Vamos usar esta seção para responder as seguintes perguntas:

Qual a proporção de gênero entre os candidatos?
Qual a média das provas por tipo de escola?

In [ ]:
# grupos
df.groupby(by = ['TP_SEXO']).groups

In [ ]:
# proporção de gênero
df.groupby(by = ['TP_SEXO'])[idCandidato].count()

De forma análoga poderíamos usar a função .value_counts(), que retorna a distribuição de frequências da variável.

In [ ]:
df.TP_SEXO.value_counts()

Agora vamos criar uma nova coluna no nosso dataset original, para mapear os valores numéricos em suas respectivas categorias. Seguidamente vamos verificar a nota média nas provas por tipo de escola.

In [ ]:
df['TP_ESCOLA_CAT'] = df.TP_ESCOLA.map({1 : 'Não respondeu', 2: 'Pública', 3: 'Privada'})

In [ ]:
df.groupby(by = ['TP_ESCOLA_CAT'])[provas].agg(np.mean).sort_values(by = ['NU_NOTA_MT'], ascending = False)

In [ ]:
# A maior parcela dos alunos são da categoria Não respondeu.
np.multiply(df['TP_ESCOLA_CAT'].value_counts(normalize = True), 100).round(2)

Para não comprometer nossa análise vamos remover a categoria Não respondeu.

In [ ]:
# distribuição de frequências em termos absolutos
df.query('TP_ESCOLA_CAT != "Não respondeu"')['TP_ESCOLA_CAT'].value_counts(normalize = False)

In [ ]:
# distribuição de frequências em termos relativos
np.multiply(df.query('TP_ESCOLA_CAT != "Não respondeu"')['TP_ESCOLA_CAT'].value_counts(normalize = True),
            100)\
.round()

Para evitar repetição e construir um código mais limpo poderíamos fazer da seguinte forma:

In [ ]:
subset_tp_escola = df[df.TP_ESCOLA_CAT != 'Não respondeu']
# distribuição de frequência em termos relativos
display(subset_tp_escola.TP_ESCOLA_CAT.value_counts(normalize = True))
# distribuição de frequência em termos absolutos
display(subset_tp_escola.TP_ESCOLA_CAT.value_counts(normalize = False))

Agora vamos obter a média das notas por tipo de escola:

In [ ]:
subset_tp_escola\
.groupby(by = ['TP_ESCOLA_CAT'])[provas]\
.agg(np.mean)\
.sort_values(by = ['NU_NOTA_MT'], ascending = False)

Utilizaremos agora mais de uma coluna para agrupar a nossa amostra e aplicaremos várias funções de agregação.

In [ ]:
# análise apenas para a prova de matemática
(
    subset_tp_escola
    .groupby(by = ['TP_ESCOLA_CAT', 'TP_LOCALIZACAO_ESC'], as_index = False)['NU_NOTA_MT']
    .agg([np.mean])
)

In [ ]:
# análise para todas as provas
(subset_tp_escola
 .groupby(by = ['TP_ESCOLA_CAT', 'TP_LOCALIZACAO_ESC'])[provas]
 .agg([np.min, np.mean, np.median, np.std, np.max])
 .transpose()
)

Com a função .agg() conseguimos também aplicar funções de agregação distintas para cada coluna:

In [ ]:
(
    subset_tp_escola
    .groupby(by = ['TP_ESCOLA_CAT'])
    .agg(
        {
            'NU_NOTA_MT': [np.mean, np.median],
            'NU_NOTA_CN': [np.min, np.max]
        }

        )
)

# Join e concat

No Pandas `pd.merge()` é uma função utilizada para unir datasets por meio de uma coluna que seja comum para ambos. Frequentemente precisamos consultar dados em várias tabelas e consolidar em uma tabela final. Vamos imaginar que precisamos construir uma tabela com dados cadastrais dos clientes de uma empresa. Provável que estes dados estejam espalhados ao longo de várias tabelas.

Exemplo:

* tb_cadastro
* tb_celular
* tb_endereco
* tb_tempo_relacionamento

Comum a todas as tabelas será provavelmente alguma coluna como o id do cliente (primary key). Desta forma, para consolidar todas as informações cadastrais dos clientes em uma única tabela precisaríamos percorrer várias tabelas e fazer um join. Ocorre ainda a possibilidade de termos que consolidar outros dados por meio de transformação de variáveis e consolidar nesta tabela cadastral, pense por exemplo se a necessidade fosse conta quantas compras este consumidor fez no último mês.


Vejamos os principais parâmetros da função merge (neste caso em ambos os datasets, temos que o nome da coluna em comum seja o mesmo):

```python
df_left.merge(df_right, on = 'column', how = 'left|right|outer|inner')
```

Caso o nome da coluna em comum seja distinta entre os datasets então teríamos:


```python
df_left.merge(df_right, left_on = 'column_left', right_on = 'column_right', how = 'inner')
```


Tipos de joins

<img src = 'https://datascience.quantecon.org/_images/merge_venns.png' width = 400 />


Vejamos agora exemplos práticos.

In [ ]:
customer_1 = pd.DataFrame({
    'id_customer': [1, 2, 3, 4, 5],
    'occupation': ['Data Scientist', 'Economist', 'Data Scientist', 'Data Engineer', 'Data Scientist']
})

In [ ]:
customer_2 = pd.DataFrame({
    'id_customer': [2, 3, 4, 5],
    'experience': [10, 3, 2, 4]
})

In [ ]:
display(customer_1, customer_2)

In [ ]:
pd.merge(customer_1, customer_2)

In [ ]:
customer_merged = pd.merge(customer_1, customer_2, on = 'id_customer', how = 'inner')

In [ ]:
customer_merged

In [ ]:
customer_3 = pd.DataFrame({
    'id_customer': [2, 3, 5, 8],
    'State': ['BA', 'SP', 'SP', 'RJ']
})

In [ ]:
pd.merge(customer_merged, customer_3, on = 'id_customer')

In [ ]:
customer_4 = pd.DataFrame({
    'id_customer': [2, 2, 2, 5, 5],
    'order_date': [2019, 2020, 2022, 2017, 2022]
})

In [ ]:
pd.merge(customer_merged, customer_4)

In [ ]:
customer_5 = pd.DataFrame({
    'customer_number': [1, 2, 3, 4, 5, 10, 11, 12, 13],
    'age': [40, 33, 27, 35, 39, 44, 25, 23, 29]
})

In [ ]:
pd.merge(customer_1, customer_5, left_on = 'id_customer', right_on = 'customer_number', how = 'inner')\
.drop(columns = 'customer_number')

In [ ]:
customer_1_modified = customer_1.set_index('id_customer')

In [ ]:
customer_2_modified = customer_2.set_index('id_customer')

In [ ]:
pd.merge(
    customer_1_modified,
    customer_2_modified,
    left_index = True,
    right_index = True,
)\
.reset_index()

In [ ]:
customer_1_modified.join(customer_2_modified, how = 'inner')

In [ ]:
order = pd.DataFrame({'id_customer': [3, 5],
                      'order_name': ['Book', 'Car']
                     })

In [ ]:
customer_1

In [ ]:
pd.merge(customer_1, order, how = 'inner')

In [ ]:
pd.merge(order, customer_1, how = 'left')

In [ ]:
pd.merge(order, customer_1, how = 'right').fillna(0)

In [ ]:
pd.merge(order, customer_1, how = 'outer')

Agora vamos voltar aos dados do ENEM e fazer um join da nossa base com a base que possui informações socioeconômicas.

In [ ]:
economic_data = pd.read_csv('dados_enem_2021_BA_questoes_socieconomicas.csv')

In [ ]:
economic_data.head()

In [ ]:
df.shape

In [ ]:
df_merged = df.merge(economic_data, on = 'NU_INSCRICAO', how = 'inner')

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

# Principais funções e comandos do Pandas

**Insepação de dados**

```python
df.head()

df.tail()

df.shape

df.info()

df.dtypes
```

**Manipulação de dados**
```python

# seleção de colunas

df.iloc[:, :]

df.loc[:, :]

# criação de colunas
df['new_column_1'] = (df['old_column'] / df['A'] * 100)
df_new = df.assign(new_column_2 = df['old_column'] * 5)

# remoção de colunas
df.drop(columns = ['column_1', 'column_2'])

# filtro

df.query('(column_A == 1) & (column_B != "value")')

df[(df.column_A == 1) | (df.column_B != 'value')]

df.loc[(df.column_A <= 2) & (df.column_C > 10)]



```



**Leitura de dados**

```python

pd.read_csv(filepath, sep, header, index_col, skiprows, na_values)

```

**Sumarização de dados**
```python
df.describe(include = 'all')
```

**Visualização de dados**

```python

# bar plot
df.column.value_counts().plot(kind = 'bar')

# pie plot
df.column.value_counts().plot(kind = 'pie')

# box plot
df.column.plot(kind = 'box')
```




